![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev&file=03c+-+model+signature.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/03c%20-%20model%20signature.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2F03c%2520-%2520model%2520signature.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/03c%20-%20model%20signature.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/03c%20-%20model%20signature.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 03c - Investigate Model Signature
In the 03c notebook the pipeline export a model from BigQuery and (conditionally) update an endpoint with this model.  The model signature is not the expected name of "serving_default" so prediction request fail.  In the notebook this is worked around by using a raw prediction request including the signature name of "predict".

The goal of this investigation is to retrieve the signature name and if it does not match "serving_default" update it.  

In [66]:
MODEL_URI = 'gs://statmike-demo3/fraud/models/03c'

DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'

In [67]:
import tensorflow as tf

In [68]:
tf1_model = tf.saved_model.load(export_dir = MODEL_URI)

In [69]:
tf1_model.signatures.keys()

KeysView(_SignatureMap({'predict': <ConcreteFunction pruned(V6, V5, V27, V25, V19, Amount, V16, V9, V3, V14, V10, V2, V12, V8, V18, V15, V7, V23, V26, V20, V21, V4, V24, V11, Time, V17, V22, V1, V28, V13) at 0x7F216FA2C810>}))

In [70]:
tf1_model.signatures['predict'].structured_outputs

{'logits': <tf.Tensor 'dnn/logits/BiasAdd:0' shape=(None, 1) dtype=float32>,
 'classes': <tf.Tensor 'dnn/head/predictions/hash_table_Lookup/LookupTableFindV2:0' shape=(None, 1) dtype=string>,
 'all_class_ids': <tf.Tensor 'dnn/head/predictions/Tile:0' shape=(None, 2) dtype=int32>,
 'all_classes': <tf.Tensor 'dnn/head/predictions/Tile_1:0' shape=(None, 2) dtype=string>,
 'class_ids': <tf.Tensor 'dnn/head/predictions/ExpandDims:0' shape=(None, 1) dtype=int64>,
 'logistic': <tf.Tensor 'dnn/head/predictions/logistic:0' shape=(None, 1) dtype=float32>,
 'probabilities': <tf.Tensor 'dnn/head/predictions/probabilities:0' shape=(None, 2) dtype=float32>}

---

In [71]:
tf1_model.signatures

_SignatureMap({'predict': <ConcreteFunction pruned(V6, V5, V27, V25, V19, Amount, V16, V9, V3, V14, V10, V2, V12, V8, V18, V15, V7, V23, V26, V20, V21, V4, V24, V11, Time, V17, V22, V1, V28, V13) at 0x7F216FA2C810>})

In [82]:
t_spec = tf.TensorSpec([None,1], tf.float32)
c_func = tf1_model
signatures = {'serving_default': c_func}

In [83]:
tf.saved_model.save(tf1_model, 'tf1_model', signatures = signatures)

ValueError: Expected a TensorFlow function to generate a signature for, but got <tensorflow.python.training.tracking.tracking.AutoTrackable object at 0x7f217c07aed0>. Only `tf.functions` with an input signature or concrete functions can be used as a signature.

In [ ]:
import tensorflow as tf
m = tf.saved_model.load("tf2-preview_inception_v3_classification_4")
print(m.signatures) # _SignatureMap({}) - Empty
t_spec = tf.TensorSpec([None,None,None,3], tf.float32)
c_func = m.__call__.get_concrete_function(inputs=t_spec)
signatures = {'serving_default': c_func}
tf.saved_model.save(m, 'tf2-preview_inception_v3_classification_5', signatures=signatures)

# Test new model
m5 = tf.saved_model.load("tf2-preview_inception_v3_classification_5")
print(m5.signatures) # _SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, inputs) at 0x17316DC50>})

---
test a tf 2.x model:

In [20]:
tf2_model = tf.saved_model.load(export_dir = 'gs://statmike-demo3/fraud/models/05')

In [21]:
tf2_model.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F216FF60ED0>}))

In [31]:
tf2_model.signatures['serving_default']

<ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F216FF60ED0>

---

In [44]:
tf.saved_model.save(tf1_model, './tf1_model')

INFO:tensorflow:Assets written to: ./tf1_model/assets


In [45]:
!saved_model_cli show --all --dir tf1_model


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

Defined Functions:

In [50]:
tf1_model = tf.compat.v1.saved_model.load(sess = tf.compat.v1.Session(), tags = [tf.compat.v1.saved_model.tag_constants.SERVING], export_dir = 'gs://statmike-demo3/fraud/models/03c')

INFO:tensorflow:Restoring parameters from gs://statmike-demo3/fraud/models/03c/variables/variables


In [51]:
tf.saved_model.save(tf1_model, './tf1_model')

ValueError: Expected a Trackable object for export, got meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "ArgMax"
      input_arg {
        name: "input"
        type_attr: "T"
      }
      input_arg {
        name: "dimension"
        type_attr: "Tidx"
      }
      output_arg {
        name: 

In [29]:
!ls

'03c - model signature.ipynb'			   dataproc_lr.ipynb
'08a - Vertex AI - Pipelines - BQML 02+03.ipynb'  'forecasting master.ipynb'
'AutoML to Cloud Run.ipynb'			   model
'Custom Managed Notebook.ipynb'			   pyspark_ml_lr_dataproc.ipynb
'Notes - Just Notes.ipynb'			   sample_kmeans_data.txt
'aa - TFX on Vertex AI - Interactive.ipynb'	  'sparkml gmm test.ipynb'
 dataproc_gmm.ipynb				   temp


In [36]:
!saved_model_cli show --dir 'gs://statmike-demo3/fraud/models/03c'

The given SavedModel contains the following tag-sets:
'serve'


In [37]:
!saved_model_cli show --dir 'gs://statmike-demo3/fraud/models/03c' --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "predict"


In [38]:
!saved_model_cli show --dir 'gs://statmike-demo3/fraud/models/03c' --tag_set serve --signature_def predict

The given SavedModel SignatureDef contains the following input(s):
  inputs['Amount'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_29:0
  inputs['Time'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder:0
  inputs['V1'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_1:0
  inputs['V10'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_10:0
  inputs['V11'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_11:0
  inputs['V12'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_12:0
  inputs['V13'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_13:0
  inputs['V14'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_14:0
  inputs['V15'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1)
      name: Placeholder_15:0
  inputs['V16'] tensor_info:
      dtype: D

In [39]:
!saved_model_cli show --dir 'gs://statmike-demo3/fraud/models/03c' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['Amount'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_29:0
    inputs['Time'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder:0
    inputs['V1'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_1:0
    inputs['V10'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_10:0
    inputs['V11'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_11:0
    inputs['V12'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_12:0
    inputs['V13'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: Placeholder_13:0
    inputs['V14'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
      